# Desmascarando Robôs
### CRISP-DM Cycle 1
---

Imagine um mercado online, um palco digital onde diversos leilões se desenrolam a cada segundo. Neste ambiente, participantes do mundo inteiram lançam seus lances em busca de objetos desejados, desde joias até equipamentos tecnológicos. No entanto, nem todos os jogadores neste campo são humanos; alguns são robôs programados para manipular os resultados dos leilões.

Seu desafio é se aprofundar nesses dados, explorar as camadas de atividade nos leilões e conseguir construir um modelo que saiba muito bem diferenciar humanos de robôs.

> Disclaimer: This is a fictional bussiness cas

## 0. PREPARATION

### 0.1 Settings

In [1]:
# Settings imports
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Load .env file
env_path = "../.env"
load_dotenv(dotenv_path=env_path)

# Seed
seed = int(os.getenv("SEED"))

# Add path
path = os.getenv("HOMEPATH")

# Add path to sys.path
sys.path.append(path)

In [2]:
from catboost import CatBoostClassifier

In [3]:
from helper.classes.FeatureEngineering import FeatureEngineering
from helper.classes.Predictions import Predictions

/Users/guttofranca/Repos/desmascarando-robos/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.2 Data

**Train e Test**

- **id_participante**: Identificador único do participante
- **conta_pagamento**: Conta de pagamento associada ao participante (com o valor ocultado) # Não será utilizada
- **endereco**: Endereço postal do participante # Não será utilizada
- **resultado**: A variável alvo que identifica se o participante é um robô ou um humano. (Robô = 1 e Humano = 0). (*target*)

- **Robôs Confirmados**: Participantes com provas claras de atividades fraudulentas, resultando em banimento da plataforma. São rotulados como robôs no conjunto de dados (resultado = 1).

- **Robôs Suspeitos**: Participantes com atividades atípicas ou estatísticas que superam a média, mas sem provas definitivas de fraude. A classificação deles como robôs é incerta.

**Lances**

- **id_lance**: Identificador único do lance
- **id_participante**: Identificador único do participante
- **leilao**: Identificador único do leilão 
- **mercadoria**: A categoria da mercadoria leiloada
- **dispositivo**: O dispositivo utilizado pelo visitante
- **tempo**: O tempo que o lance foi feito
- **pais**: O país que o IP pertence
- **ip**: O IP do participante
- **url**: A URL de onde o participante foi referido

## 1. Modelling

### 1.1 Loading Data

In [4]:
X_train = pd.read_feather(path + "/data/processed/X_train.feather")
X_test = pd.read_feather(path + "/data/processed/X_test.feather")
X_val = pd.read_feather(path + "/data/processed/X_val.feather")

y_train = pd.read_pickle(path + "/data/processed/y_train.pkl")
y_test = pd.read_pickle(path + "/data/processed/y_test.pkl")
y_val = pd.read_pickle(path + "/data/processed/y_val.pkl")

X = pd.concat([X_train, X_test, X_val], axis=0)
y = pd.concat([y_train, y_test, y_val], axis=0)

test = pd.read_feather(path + "/data/processed/test.feather")
fe = FeatureEngineering()
test = fe.perform_transformations(test)


X.drop(
    columns=[
        "pais",
        "url",
        "endereco",
        "dispositivo",
        "leilao",
        "periodo_dia",
        "mercadoria",
        "conta_pagamento",
    ],
    inplace=True,
)

In [5]:
feature_transformations = {
    "log": [
        "contagem_participante",
        "contagem_leilao",
        "contagem_conta_pagamento",
        "frequencia_dispositivo",
    ],
    # "one_hot": [
    #    "dispositivo",
    #    "leilao",
    #    "periodo_dia",
    #    "mercadoria",
    #    "conta_pagamento",
    # ],
    "ordinal": ["ip_classe"],
    # "hashing": ["pais", "url", "endereco"],
    "min_max_scaler": [
        "hora_sin",
        "hora_cos",
        "minuto_sin",
        "minuto_cos",
        "segundo_sin",
        "segundo_cos",
    ],
    "robust_scaler": ["hora", "minuto", "segundo"],
}


proportion = float(len(y_train[y_train == 0])) / len(y_train[y_train == 1])

model = CatBoostClassifier(
    iterations=905,
    depth=9,
    learning_rate=0.0032193578881126797,
    random_strength=0.05022200281503645,
    bagging_temperature=0.5648259544017337,
    border_count=1,
    scale_pos_weight=proportion,
    random_state=seed,
    verbose=0,
)

In [6]:

final_pipeline = Predictions(feature_transformations, model)
final_pipeline.fit(X, y)
test = final_pipeline.add_predictions(test)

Training model CatBoostClassifier...
Starting fit_transform...
Transformers configured.
ColumnTransformer created.
fit_transform completed.
Model CatBoostClassifier trained.
Adding predictions to DataFrame...
Starting transform...
transform completed.
Predictions added to DataFrame.


In [7]:
test.head()

,id_participante,conta_pagamento,endereco,id_lance,leilao,mercadoria,dispositivo,tempo,pais,ip,...,mercadoria_mais_frequente,dispositivo_mais_usado,total_dispositivos,total_paises,pais_mais_frequente,total_ips,ip_mais_frequente,total_urls,url_mais_frequente,predicao
0,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2394993,8gshg,joias,phone469,1970-04-22 11:33:30.789473684,my,233.253.145.73,...,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu,0.510794
1,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2401931,8gshg,joias,phone46,1970-04-22 11:33:46.315789473,my,155.48.112.213,...,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu,0.535670
2,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2406130,ca59r,joias,phone46,1970-04-22 11:33:56.000000000,my,28.105.143.62,...,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu,0.381094
3,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2452472,8gshg,joias,phone13,1970-04-22 11:35:52.421052631,my,56.49.2.96,...,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu,0.516026
4,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2456348,8gshg,joias,phone524,1970-04-22 11:36:03.105263157,my,95.116.232.134,...,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu,0.516026


In [8]:
test[['id_participante', 'predicao']].to_csv(path + "/data/processed/predicao.csv", index=False)